In [76]:
import pennylane as qml
from pennylane import numpy as pnp
from scipy.optimize import differential_evolution, minimize
from scipy.stats.qmc import Halton
import os
import json
import numpy as np
from datetime import datetime
import time
from qiskit.quantum_info import SparsePauliOp
from susy_qm import calculate_Hamiltonian

In [200]:
shots = 1024
potential = 'AHO'
cutoff = 8

In [201]:
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues, eigenvectors = np.linalg.eig(H)
sorted_indices = np.argsort(eigenvalues)
lowest_indices = sorted_indices[:3]

#eigenvalues = eigenvalues[lowest_indices]
#eigenvectors = eigenvectors[lowest_indices]

min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [203]:
np.sort(eigenvalues)

array([3.20101100e-02+0.j, 1.68015552e+00+0.j, 1.83352558e+00+0.j,
       3.96337391e+00+0.j, 4.00985462e+00+0.j, 4.03692369e+00+0.j,
       8.26209394e+00+0.j, 8.77647491e+00+0.j, 4.25280809e+01+0.j,
       4.34333354e+01+0.j, 5.53066896e+01+0.j, 5.62063259e+01+0.j,
       3.81878539e+02+0.j, 3.82821281e+02+0.j, 4.08644293e+02+0.j,
       4.09587044e+02+0.j])

In [193]:
vec = min_eigenvector

non_zero_indices = np.where(np.abs(vec) > 1e-6)[0]
print("Non-zero indices:", non_zero_indices)
print("Number of non-zero entries:", len(non_zero_indices))


Non-zero indices: [8]
Number of non-zero entries: 1


In [194]:
# Implements the weighted subspace search VQE [1810.09434]
num_layers = 1
params_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)

# Weight vector - weights the different eigenstates in the cost function so
# that it's the lowest ones that are found
w = np.arange(num_qubits, 0, -1)

# This ansatz works well enough, it's not the same as the one in the paper though
wires = range(num_qubits)
def ansatz(params, state_idx=0, wires=wires):
    params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
    qml.PauliX(wires=wires[state_idx])
    qml.StronglyEntanglingLayers(weights=params, wires=wires)

In [195]:
#single_cost = qml.ExpvalCost(ansatz, H, dev)
dev = qml.device('default.qubit', wires=num_qubits, shots=shots)

@qml.qnode(dev)
def cost_function(params, state_idx=0):
    ansatz(params, state_idx)
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [196]:
# The full cost - computes single_cost for each starting state
def total_cost(params):
    cost = 0
    for state_idx in range(num_qubits):
        cost += w[state_idx] * cost_function(params, state_idx=state_idx)
    return cost

In [204]:
bounds = [(0, 2 * np.pi) for _ in range(np.prod(params_shape))]

max_iter = 10000
popsize = 20
tol = 1e-3
abs_tol = 1e-3

seed = (os.getpid() * int(time.time())) % 123456789
halton_sampler = Halton(d=np.prod(params_shape), seed=seed)
halton_samples = 2 * np.pi * halton_sampler.random(n=popsize)

x0 = np.random.uniform(low=0, high=2*np.pi, size=np.prod(params_shape))

print("Starting SSVQE optimization")
res = differential_evolution(total_cost, bounds=bounds, maxiter=max_iter, tol=tol, atol=abs_tol, popsize=popsize, init=halton_samples, seed=seed)
#res = minimize(
#        total_cost,
#        x0,
#        method= "COBYLA",
#        options= {'maxiter':max_iter}
 #   )

print("Optimization complete")

Starting SSVQE optimization
Optimization complete


In [205]:
energies = []
for state_idx in range(num_qubits):
    energies.append(cost_function(res.x, state_idx=state_idx))

In [206]:
energies

[np.float64(22.61711286440554),
 np.float64(36.68645806610892),
 np.float64(57.53413334721161),
 np.float64(69.44573388514824)]

In [12]:
run = {
    "starttime": starttime,
    "potential": potential,
    "cutoff": cutoff,
    "exact_eigenvalues": [x.real.tolist() for x in eigenvalues],
    "ansatz": "StronglyEntanglingLayers-1layer",
    "optimized_energies": [float(e) for e in optimized_energies],
    "params": optimized_params.tolist(),
    "success": res.success,
    "iterations": res.nit,
    #"total_run_time": str(res.execution_time),
}

run


{'starttime': '2025-01-15_16-27-32',
 'potential': 'AHO',
 'cutoff': 16,
 'exact_eigenvalues': [-0.0011669756805109488,
  1.6774941961815084,
  1.6863812488130043],
 'ansatz': 'StronglyEntanglingLayers-1layer',
 'optimized_energies': [10.807017657197598,
  3.1084260789096345,
  19.897834826249625],
 'params': [3.294735292376345,
  3.2738522804993386,
  5.471680050301552,
  3.589569219946922,
  0.011524585468796644,
  3.420167721770964,
  3.724238495511021,
  6.168231047696654,
  2.546434906556768,
  0.08538208621398713,
  5.510965088043173,
  2.905136477976369,
  2.9457995899834133,
  0.08626892110786333,
  4.1533874592583775],
 'success': False,
 'iterations': 500}

In [ ]:
path = os.path.join(base_path, "{}_{}.json".format(potential, cutoff))
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)

print(f"Results saved to {path}")